In [ ]:
!pip install tensorflow_text
import tensorflow_text as text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
# Import packages for data set preprocessing, feature engineering, and model training
import pandas as pd
import numpy as np

from google.colab import drive
from glob import glob
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import tensorflow_hub as hub

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

from sklearn import decomposition, ensemble
from keras.preprocessing import text, sequence


import xgboost, numpy, textblob, string

In [ ]:
# Combine five categories into five .csv files respetively by using for-loop

drive.mount('/content/drive')
Category = ['business', 'entertainment', 'politics', 'sport', 'tech',]
root_path= '/content/drive/MyDrive/Data/bbc/'
all= '/*'
for cat in Category:
  data = pd.DataFrame()
  title_list = []
  content_list = []
  main_list=[]
  for files in glob(root_path+cat+all):
    with open(files,'r', encoding="ISO-8859-1") as f:
      content = f.readlines()
      content.remove('\n')
      title_list.append(content[0].strip('\n'))
      content1 = [i.strip('\n') for i in content[1:]]
      content_list.append(content1) 
  data['title'] = pd.Series(title_list)
  data['content'] = pd.Series(content_list)
  data['label'] = cat
  csv_path = root_path + cat+'.csv'
  data.to_csv(root_path +cat+ '.csv',header=True,index=False,encoding='utf_8_sig')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Combine 5 individual files into an entire .csv file, which is bbc_news.csv
df0 = pd.read_csv('/content/drive/MyDrive/Data/bbc/business.csv')
print(df0.shape)
df1 = pd.read_csv('/content/drive/MyDrive/Data/bbc/entertainment.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Data/bbc/politics.csv')
df3 = pd.read_csv('/content/drive/MyDrive/Data/bbc/sport.csv')
df4 = pd.read_csv('/content/drive/MyDrive/Data/bbc/tech.csv')
df0 = df0.append(df1,ignore_index=True)
print(df0.shape)
df0 = df0.append(df2,ignore_index=True)
print(df0.shape)
df0 = df0.append(df3,ignore_index=True)
print(df0.shape)
df0 = df0.append(df4,ignore_index=True)
print(df0.shape)
df0.to_csv(root_path+'bbc_news.csv',header=True,index=False,encoding='utf_8_sig')

(510, 3)
(896, 3)
(1313, 3)
(1824, 3)
(2225, 3)


In [ ]:
# Read data of bbc_news
bbc_news_df = pd.read_csv('/content/drive/MyDrive/Data/bbc/bbc_news.csv')
bbc_news_df.head()

,title,content,label
0,Ask Jeeves tips online ad revival,['Ask Jeeves has become the third leading onli...,business
1,UK economy facing 'major risks',['The UK manufacturing sector will continue to...,business
2,US interest rate rise expected,"[""US interest rates are expected to rise for t...",business
3,Ex-Boeing director gets jail term,['An ex-chief financial officer at Boeing has ...,business
4,Bank set to leave rates on hold,['UK interest rates are set to remain on hold ...,business


In [ ]:
mainDF = pd.DataFrame()
mainDF['text'] = bbc_news_df['content']
mainDF['label'] = bbc_news_df['label']

In [ ]:
mainDF.head()

,text,label
0,['Ask Jeeves has become the third leading onli...,business
1,['The UK manufacturing sector will continue to...,business
2,"[""US interest rates are expected to rise for t...",business
3,['An ex-chief financial officer at Boeing has ...,business
4,['UK interest rates are set to remain on hold ...,business


In [ ]:
from sklearn.utils import shuffle
mainDF = shuffle(mainDF)

In [ ]:
mainDF.head()

,text,label
1739,['Ireland fly-half Ronan O\'Gara hailed his si...,sport
1633,['Chelsea manager Jose Mourinho expects the Ch...,sport
660,['Quirky comedy Sideways was named the best fi...,entertainment
494,['The Jamaican government is regaining control...,business
1387,['Liverpool may launch an Â£8m January bid for...,sport


In [ ]:
mainDF['Category']=preprocessing.LabelEncoder().fit_transform(mainDF['label'])
mainDF.head()

,text,label,Category
1739,['Ireland fly-half Ronan O\'Gara hailed his si...,sport,3
1633,['Chelsea manager Jose Mourinho expects the Ch...,sport,3
660,['Quirky comedy Sideways was named the best fi...,entertainment,1
494,['The Jamaican government is regaining control...,business,0
1387,['Liverpool may launch an Â£8m January bid for...,sport,3


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mainDF['text'],mainDF['Category'])

In [ ]:
X_train.head(4)

81      ['China has ordered a halt to construction wor...
206     ['German football club and former European cha...
1828    ['A blind student has developed software that ...
366     ['UK retail sales fell in December, failing to...
Name: text, dtype: object

In [ ]:
y_train.head()

81      0
206     0
1828    4
366     0
2080    4
Name: Category, dtype: int64

In [ ]:
# convert integers to dummy variables (i.e. one hot encoded)
from keras.utils import np_utils
dummy_y = np_utils.to_categorical(y_train)
dummy_y_test = np_utils.to_categorical(y_test)

In [ ]:
dummy_y.shape

(1668, 5)

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.843517  , -0.5132728 , -0.8884575 , ..., -0.74748886,
        -0.7531474 ,  0.91964495],
       [-0.87208354, -0.50543964, -0.94446677, ..., -0.858475  ,
        -0.7174535 ,  0.88082975]], dtype=float32)>

In [ ]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9911088]], dtype=float32)

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l= tf.keras.layers.Dense(32, activation='relu')(l)
l = tf.keras.layers.Dense(5, activation='softmax', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
len(X_train)

1668

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, dummy_y, epochs=5)

Epoch 1/5
53/53 [==============================] - 647s 12s/step - loss: 0.4875 - accuracy: 0.3273
Epoch 2/5
53/53 [==============================] - 650s 12s/step - loss: 0.4126 - accuracy: 0.5396
Epoch 3/5
53/53 [==============================] - 657s 12s/step - loss: 0.3608 - accuracy: 0.6415
Epoch 4/5
53/53 [==============================] - 652s 12s/step - loss: 0.3192 - accuracy: 0.7296
Epoch 5/5
53/53 [==============================] - 659s 12s/step - loss: 0.2889 - accuracy: 0.7656


In [ ]:
# evaluate the model
loss, accuracy = model.evaluate(X_train, dummy_y, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 83.453238
